In [2]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping


# for math and calculating time and shit
import time
import math
import random

# To import a behavior agents for controlling car
from agents.navigation.basic_agent import BasicAgent
from agents.navigation.behavior_agent import BehaviorAgent

In [3]:
# connect to the sim, create the world
client = carla.Client('localhost', 2000)

# the town we've been using is town04, feel free to load different towns, goes to 012 i believe
client.load_world('Town04')

#define environment/world and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

# function for clearing da world
def clear_world():
    # clears all vehicles + sensors and sets them to None
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
        actor = None
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
        sensor = None
        

In [6]:
# vehicle setup

# TODO: turn this into a Vehicle class, and create a TrafficManager class to handle the fleet
# collision sensor setup (for dealing with crashes)
def setup_collision_sensor(vehicle, callback):
    # Get the blueprint for the collision sensor
    blueprint_library = vehicle.get_world().get_blueprint_library()
    collision_bp = blueprint_library.find('sensor.other.collision')

    # Set the transform relative to the vehicle
    transform = carla.Transform(carla.Location(x=1.5, z=2.4))  # Front of the vehicle

    # Spawn the collision sensor and attach it to the vehicle
    collision_sensor = vehicle.get_world().spawn_actor(collision_bp, transform, attach_to=vehicle)

    # Listen for collision events
    collision_sensor.listen(lambda event: callback(event))

    return collision_sensor

# function to spawn a vehicle. Will move simulator camera to car position as well.
def spawn_vehicle(pointNumber=1):#look for a blueprint of Mini car
    global start_point
    vehicle_bp = world.get_blueprint_library().filter('*model3*')

    #spawn a car in a random location
    
    start_point = spawn_points[pointNumber]
    vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

    # move simulator view to the car
    spectator = world.get_spectator()
    start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
    spectator.set_transform(start_point)
    return vehicle

# sets up the cameras
def setup_cameras(vehicle):
    world = vehicle.get_world()
    blueprint_library = world.get_blueprint_library()
    
    # Semantic camera setup
    camera_bp = blueprint_library.find('sensor.camera.semantic_segmentation')
    camera_bp.set_attribute('image_size_x', '640')
    camera_bp.set_attribute('image_size_y', '480')
    camera_bp.set_attribute('fov', '90')
    camera_sem = world.spawn_actor(
        camera_bp,
        carla.Transform(carla.Location(z=1.3, x=1.4)),
        attach_to=vehicle
    )
    
    # RGB camera setup
    camera_bp = blueprint_library.find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', '640')
    camera_bp.set_attribute('image_size_y', '480')
    camera_bp.set_attribute('fov', '90')
    camera_rgb = world.spawn_actor(
        camera_bp,
        carla.Transform(carla.Location(z=1.3, x=1.4)),
        attach_to=vehicle
    )

    # Callbacks for camera data
    def sem_callback(image, data_dict):
        image.convert(carla.ColorConverter.CityScapesPalette)
        data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    
    def rgb_callback(image, data_dict):
        data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

    camera_data = {'sem_image': np.zeros((480, 640, 4)), 'rgb_image': np.zeros((480, 640, 4))}
    
    camera_sem.listen(lambda image: sem_callback(image, camera_data))
    camera_rgb.listen(lambda image: rgb_callback(image, camera_data))
    
    return camera_sem, camera_rgb, camera_data

def print_vehicle_controls(vehicle):
    control = vehicle.get_control()
    normalized_steer = (control.steer + 1) /2
    normalized_throttle = (control.throttle)
    normalized_brake = (control.brake)
    print("Steer: {:.3f}, Throttle: {:.3f}, Brake: {:.3f}".format(normalized_steer, normalized_throttle, normalized_brake))


def setAgent(vehicle):

    # start a basic agent for the car
    agent = BasicAgent(vehicle)

    # give it a normal profile
    agent = BehaviorAgent(vehicle, behavior='normal') # maybe set to aggressive?

    destination = spawn_points[0].location # for now we just take the second spawn point
                                  # later use a CARLA location at self.x + 10 or something

    agent.set_destination(destination)

    #lights always on
    vehicle.set_light_state(carla.VehicleLightState.LowBeam)
    return agent


def on_collision(event):
    print("Collision detected: Actor %s hit %s" % (event.other_actor.type_id, event.actor.type_id))
    # Stop the vehicle (or apply any other logic you deem necessary)
    respawn_car()
    
def respawn_car(spawnPoint=None):
    global vehicle, agent, collision_sensor, camera_sem, camera_rgb, camera_data
    # Destroy existing actors if they exist
    
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    if vehicle is not None:
        vehicle.destroy()
        vehicle = None
    if collision_sensor is not None:
        collision_sensor.destroy()
        collision_sensor = None
    if camera_sem is not None:
        camera_sem.destroy()
        camera_sem = None
    if camera_rgb is not None:
        camera_rgb.destroy()
        camera_rgb = None
    
    
    # if spawnpoint is unspecified, spawn new vehicle at random place
    
    if (spawnPoint):
        random_spawnpoint = spawnPoint
    
    print("Vehicle spawned at spawn point: ", random_spawnpoint)
    random_spawnpoint = random.randint(0, 10)  # Both 1 and 10 are included
    vehicle = spawn_vehicle(random_spawnpoint)
    
    # sometimes the vehicle isn't spawned correctly and 
    # other threads try to read it and then it gets fucked
    if vehicle is None:
        raise Exception("Failed to spawn vehicle")
        time.sleep(1)

    # delay for a second just in case
    time.sleep(1)
    
    # Setup collision sensor
    collision_sensor = setup_collision_sensor(vehicle, on_collision)

    # Setup cameras
    camera_sem, camera_rgb, camera_data = setup_cameras(vehicle)

    # Setup or reset the agent
    agent = setAgent(vehicle)

    print("New vehicle and sensors spawned. Continuing simulation...")

In [10]:
# helper functions for the main loop

def get_random_waypoint(world):
    map = world.get_map()
    # Generate waypoints every 2 meters
    waypoints = map.generate_waypoints(distance=2.0)
    return random.choice(waypoints)

def distance(location1, location2):
    return math.sqrt((location1.x - location2.x)**2 + (location1.y - location2.y)**2 + (location1.z - location2.z)**2)

# returns the openCV objects of rgb/segmented images from CARLA
def grab_pictures():
#     print_vehicle_controls(vehicle)
        
#     steering_wheel_direction = control.steer
    rgb_im = camera_data['rgb_image']
    sem_im = camera_data['sem_image']

    # display images
    im_h = cv2.hconcat([rgb_im,sem_im])
    cv2.imshow('2 cameras', im_h)
    
    return rgb_im, sem_im

In [9]:
# initialize global variables and create the car
    
vehicle = None
agent = None
collision_sensor = None
camera_sem = None
camera_rgb = None
camera_data = None
start_point = None

respawn_car()

10
New vehicle and sensors spawned. Continuing simulation...


In [ ]:
# function to capture CARLA's segmented images.
# as of now, ./out_sem/rgb and ./out_sem/sem folder for it to export properly.


# TODO: will refactor the code to make it object-oriented and implement thread locks
last_capture_time = time.time()
interval = 7.5 # set interval to 10 seconds
sharp_steer_interval = 0.1
light_steer_interval = 0.75


    
current_waypoint = None

if vehicle is None:
    respawn_car()
while True:
    
    # advance the world
    world.tick()

    if vehicle is not None and agent is not None:
        try:
            control = agent.run_step()

            vehicle.apply_control(control)
            current_time = time.time()
            control = vehicle.get_control()

            # note try to factor this out later lol
            # on light turn, delay 0.5
            if abs(control.steer * 100) >= 5 and abs(control.steer * 100) < 10:


                if current_time - last_capture_time >= light_steer_interval:
                    rgb_im, sem_im = grab_pictures()
                    # grab the time
                    time_grab = time.time_ns()
                    # normalize controls to between 0 and 100, with 50 being straight or nothing
                    normalized_steer = (control.steer + 1) /2
                    normalized_throttle = (control.throttle)
                    normalized_brake = (control.brake)
                    cv2.imwrite('out_sem/rgb/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), rgb_im)
                    cv2.imwrite('out_sem/sem/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), sem_im)
                    last_capture_time = current_time  # Update time

            # on sharp turn, delay 0.1
            elif abs(control.steer * 100) >= 10:
                if current_time - last_capture_time >= sharp_steer_interval:
                    # normalize controls to between 0 and 100, with 50 being straight or nothing
                    rgb_im, sem_im = grab_pictures()
                    # grab the time
                    time_grab = time.time_ns()
                    normalized_steer = (control.steer + 1) /2
                    normalized_throttle = (control.throttle)
                    normalized_brake = (control.brake)
                    cv2.imwrite('out_sem/rgb/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), rgb_im)
                    cv2.imwrite('out_sem/sem/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), sem_im)
                    last_capture_time = current_time  # Update time


            elif current_time - last_capture_time >= interval:        
                rgb_im, sem_im = grab_pictures()

                time_grab = time.time_ns()

                normalized_steer = (control.steer + 1) /2
                normalized_throttle = (control.throttle)
                normalized_brake = (control.brake)
                cv2.imwrite('out_sem/rgb/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), rgb_im)
                cv2.imwrite('out_sem/sem/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), sem_im)        
                # update time
                last_capture_time = current_time
            if cv2.waitKey(1) == ord('q'):
                break
            if agent.done():
                while True:
                    new_waypoint = get_random_waypoint(world)
                    if current_waypoint is None or distance(current_waypoint.transform.location, new_waypoint.transform.location) > 50:  # Check if new waypoint is at least 50 meters away
                        break
                print("reached waypoint, heading to: ", new_waypoint) 
                agent.set_destination(new_waypoint.transform.location)
                current_waypoint = new_waypoint
        except Exception as e:
            print("Caught RuntimeError while trying to control the vehicle:", e)
            time.sleep(1)
    

clear_world()
cv2.destroyAllWindows()

Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined


In [11]:
from predict_control import predict


# if vehicle is None:
respawn_car()
    
# see if you can keep a '3 second log' and then write to it when you crash
while True:
    
    world.tick()
    try:
        rgb_im, sem_im = grab_pictures()

        prediction = predict(sem_im, CARLA=True)

        new_steering = 2 * (prediction[0] - 0.5)
        print(f"steer: {new_steering}, throttle: {prediction[1]}, brake: {prediction[2]}")
        control = carla.VehicleControl(steer=new_steering, throttle=prediction[1], brake=0.0)
        vehicle.apply_control(control)
        current_time = time.time()
    except Exception as e:
        print(e)
        time.sleep(1)
        respawn_car()
#     control = vehicle.get_control()
    
    if cv2.waitKey(1) == ord('q'):
        break
clear_world()
cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Clarence\\Documents\\Coding\\cs141\\researchProject\\selfDrivingCar\\Self_Driving_Car\\models\\driver_model-3.pt'

In [18]:
#grab a snapshot from the camera an show in a pop-up window
img = camera_data['image']
cv2.imshow('RGB Camera',img)
cv2.waitKey(0)

113

In [82]:
# clean up after yourself

# camera.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [7]:
len(spawn_points)

10422

In [8]:
start_point = spawn_points[0]
spectator = world.get_spectator()
spectator.set_transform(start_point)

In [12]:
spectator.set_transform(carla.Transform(carla.Location(x=-1085.286377, y=3112.225830, z=356.060608), carla.Rotation(pitch=1.648070, yaw=20.234367, roll=0.000000)))

In [11]:
print(start_point)

Transform(Location(x=-1085.286377, y=3112.225830, z=356.060608), Rotation(pitch=1.648070, yaw=20.234367, roll=0.000000))
